### Vibrometer Measurements of Car
##### Measurement Summary:
- car off
- car on
- car on, music playing
- **revving car engine**

This notebook creates plots for the case of revving the car engine

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import scipy.stats as ss
from sklearn.preprocessing import MinMaxScaler
import scipy.fft
import pywt
from scipy.signal.windows import hann
from scipy import interpolate
from scipy.ndimage import uniform_filter

***
### Revving Engine
##### Load data:

In [ ]:
# Load data:
# (make sure the .ipynb file is in the same folder as the data)
column_names1 = ['Time1 (s)', 'Signal1 (m)']
df1 = pd.read_table("revcar1.txt", delimiter=r'\s+', encoding='latin-1', skiprows=5, names=column_names1)
#print(df1.head())

column_names2 = ['Time2 (s)', 'Signal2 (m)']
df2 = pd.read_table("revcar2.txt", delimiter=r'\s+', encoding='latin-1', skiprows=5, names=column_names2)
#print(df2.head())

column_names3 = ['Time3 (s)', 'Signal3 (m)']
df3 = pd.read_table("revcar3.txt", delimiter=r'\s+', encoding='latin-1', skiprows=5, names=column_names3)
#print(df3.head())

column_names4 = ['Time4 (s)', 'Signal4 (m)']
df4 = pd.read_table("revcar4.txt", delimiter=r'\s+', encoding='latin-1', skiprows=5, names=column_names4)
#print(df4.head())
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# file naming changes after revcar4.txt
column_names5 = ['Time5 (s)', 'Signal5 (m)']
df5 = pd.read_table("revcarr1.txt", delimiter=r'\s+', encoding='latin-1', skiprows=5, names=column_names5)

column_names6 = ['Time6 (s)', 'Signal6 (m)']
df6 = pd.read_table("revcarr2.txt", delimiter=r'\s+', encoding='latin-1', skiprows=5, names=column_names6)

column_names7 = ['Time7 (s)', 'Signal7 (m)']
df7 = pd.read_table("revcarr3.txt", delimiter=r'\s+', encoding='latin-1', skiprows=5, names=column_names7)

column_names8 = ['Time8 (s)', 'Signal8 (m)']
df8 = pd.read_table("revcarr4.txt", delimiter=r'\s+', encoding='latin-1', skiprows=5, names=column_names8)

column_names9 = ['Time9 (s)', 'Signal9 (m)']
df9 = pd.read_table("revcarw1.txt", delimiter=r'\s+', encoding='latin-1', skiprows=5, names=column_names9)

column_names10 = ['Time10 (s)', 'Signal10 (m)']
df10 = pd.read_table("revcarw2.txt", delimiter=r'\s+', encoding='latin-1', skiprows=5, names=column_names10)

column_names11 = ['Time11 (s)', 'Signal11 (m)']
df11 = pd.read_table("revcarw3.txt", delimiter=r'\s+', encoding='latin-1', skiprows=5, names=column_names11)

column_names12 = ['Time12 (s)', 'Signal12 (m)']
df12 = pd.read_table("revcarw4.txt", delimiter=r'\s+', encoding='latin-1', skiprows=5, names=column_names12)

##### Plot Time Series:

In [ ]:
rev_car_dataframes = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12] # add more after loading more dataframes
for i, dframe in enumerate(rev_car_dataframes):
    time_column = f"Time{i+1} (s)"
    signal_column = f"Signal{i+1} (m)"
    # Plot:
    plt.figure(figsize=(12,4))
    plt.plot(dframe[time_column], dframe[signal_column], label=f"Revving Engine Measurement {i+1} - Full Data")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude (m)")
    plt.grid(True,alpha=0.5)
    plt.title(f"Revving Car Engine Measurement {i+1} - Full Data")
    plt.tight_layout()
    plt.show()

##### Detrend and Shift Data to Zero Mean:

In [ ]:
for i, dframe in enumerate(rev_car_dataframes):
    time_column = f"Time{i+1} (s)"
    signal_column = f"Signal{i+1} (m)"
    detrended_signal_column = f"Signal{i+1} Detrended (m)"
    mean_shifted_column = f"Signal{i+1} Zero Mean (m)"
    
    # Detrend
    dframe[detrended_signal_column] = signal.detrend(dframe[signal_column], type='linear') 
    # Shift Mean to ~Zero (without normalizing)
    dframe[mean_shifted_column] = dframe[detrended_signal_column] - dframe[detrended_signal_column].mean()

    # Plot:
    plt.figure(figsize=(12,4))
    plt.plot(dframe[time_column], dframe[mean_shifted_column], label=f"Revving Engine Measurement {i+1} - Full Data")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude (m)")
    plt.grid(True,alpha=0.5)
    plt.axhline(y=0.0, label="0.0", color='red', linestyle=":")
    plt.title(f"Revving Car Engine Measurement {i+1} - Detrended and Shifted to Zero Mean")
    plt.tight_layout()
    plt.show()

##### Tapering Detrended and Shifted (DS) Time Series
- A Hann window is used for tapering

In [ ]:
def apply_hann_window(data, column):
    window = np.hanning(len(data))
    windowed_data = data[column] * window
    return windowed_data
    
for i, dframe in enumerate(rev_car_dataframes):
    tapered_data = []
    mean_shifted_column = f"Signal{i+1} Zero Mean (m)"
    hann_result = apply_hann_window(dframe, mean_shifted_column) # taper all dataframes
    tapered_data.append(hann_result)
    time_column = f"Time{i+1} (s)" # need time column for plotting
    # Plot:
    plt.figure(figsize=(12,4))
    plt.plot(dframe[time_column], dframe[mean_shifted_column], label=f"Full Data")
    plt.plot(dframe[time_column], tapered_data[-1], label=f"Tapered Data")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude (m)")
    plt.grid(True,alpha=0.5)
    plt.title(f"Revving Car Engine Measurement {i+1} - Full and Tapered DS Data")
    plt.legend()
    plt.tight_layout()
    plt.show()

##### 'percent_check' function for tapering

In [ ]:
def percent_check_tapered(original_data, tapered_data_full, padding, tolerance=0.05):
    """original_data : Original unpadded time series (array)
    tapered_data_full : Tapered time series, includes padding (array)
    padding : Amount of padding added to each end (int)
    tolerance : Acceptable error as fraction of data range (float)"""
    # Extract original from tapered data (remove padding)
    if padding > 0:
        tapered_original_region = tapered_data_full[padding:-padding]
    else:
        tapered_original_region = tapered_data_full
    
    # Direct comparison: tapered vs original
    abs_errors = np.abs(original_data - tapered_original_region)
    # normalization
    data_range = np.max(original_data) - np.min(original_data)
    normalized_errors = abs_errors / data_range
    
    # Calculate metrics
    within_tolerance = normalized_errors <= tolerance
    pct_within = (np.sum(within_tolerance) / len(within_tolerance))*100
    
    # Mean Absolute Error (MAE) metrics
    mae = np.mean(abs_errors)
    std_original = np.std(original_data)
    mae_pct_std = (mae / std_original)*100
    
    passes = pct_within >= 95
    
    results = {'padding': padding,
        'tolerance': f'{tolerance*100}%',
        'passes': passes,
        'points_within_tolerance': f'{pct_within: .4f}%',
        'max_normalized_error': f'{(np.max(normalized_errors)*100): .4f}%',
        'mean_abs_error': f'{mae: .4f}',
        'mae_as_%_of_std': f'{mae_pct_std: .4f}%',
        'num_points': len(original_data),
        'data_range': f'{data_range: .4f}'}
    
    return results
    
for i, dframe in enumerate(rev_car_dataframes):
    mean_shifted_column = f"Signal{i+1} Zero Mean (m)"
    tapered_data = []
    hann_result = apply_hann_window(dframe, mean_shifted_column) # taper all dataframes
    tapered_data.append(hann_result)
    percent_check_tapered_results = percent_check_tapered(dframe[mean_shifted_column], tapered_data[-1], padding=0, tolerance=0.05)
    print(f"dataframe: df{i+1}")
    print(percent_check_tapered_results)
    print("")

##### Subsampling DS Time Series

In [ ]:
# Subsampling the tapered time series
#Sampling Rate = Total Number of Samples / Total Time (in seconds)
def subsample(data, column, decimation_factor):
    """Subsample data using scipy.signal.decimate with anti-aliasing filter.
    data: input data e.g. df1
    column: column to subsample (e.g. "Signal1 Zero Mean (m)"), string
    decimation_factor: factor by which to reduce sampling rate"""
    num_samples=len(data[column])
    sampling_time=data.iloc[-1, 0] #get last value in first column (time (s))
    original_sampling_rate = int(num_samples/sampling_time)
    print(f"Original sampling rate for {column}: {original_sampling_rate} Hz")
    # scipy.signal.decimate applies an anti-aliasing filter automatically
    # Use a higher order filter for better anti-aliasing (default is 8)
    decimated_data = scipy.signal.decimate(data[column], decimation_factor, ftype='iir', zero_phase=True) # 
    # get new sampling rate:
    new_sr = int(original_sampling_rate / decimation_factor)
    print(f"New sampling rate for {column}: {new_sr} Hz")
    return decimated_data, new_sr, original_sampling_rate

for i, dframe in enumerate(rev_car_dataframes):
    mean_shifted_column = f"Signal{i+1} Zero Mean (m)"
    time_column = f"Time{i+1} (s)"
    subsample_result = subsample(dframe, mean_shifted_column, decimation_factor=50)
    subsampled_data = subsample_result[0]
    # sub_list = []
    # sub_list.append(subsampled_data)
    
    # Plot:
    plt.figure(figsize=(12,4))
    plt.plot(dframe[time_column], dframe[mean_shifted_column], label=f"Full Data")
    plt.plot(dframe[time_column][::50], subsampled_data, label=f"Subsampled Data") # Make sure time step is same as decimation_factor
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude (m)")
    plt.grid(True, alpha=0.5)
    plt.title(f"Revving Car Engine Measurement {i+1} - Full and Subsampled DS Data")
    plt.legend()
    #plt.xlim(4,5)
    plt.tight_layout()
    plt.show()

##### 'percent_check' function for subsampling

In [ ]:
def percent_check_scipy_decimate(data, column, step, tolerance=0.05):
    x_original = np.arange(len(data))
    y_original = data[column].values
    
    # Get subsampled series
    x_subsampled = np.arange(0, len(data), step)
    y_subsampled=scipy.signal.decimate(data[column].values, step, ftype='iir', zero_phase=True)
    
    # Interpolate subsampled data back to original timestamps
    f_interp = interpolate.interp1d(x_subsampled, y_subsampled, kind='linear', fill_value='extrapolate')
    y_reconstructed = f_interp(x_original)
    
    # Calculate absolute errors
    abs_errors = np.abs(y_original - y_reconstructed)
    
    # Calculate data range (the scale of the data)
    data_range = np.max(y_original) - np.min(y_original)
    #print("data_range:", data_range)
    # Normalize errors by data range instead of individual values
    normalized_errors = abs_errors / data_range
    
    # Test if within tolerance
    within_tolerance = normalized_errors <= tolerance
    pct_within = (np.sum(within_tolerance) / len(within_tolerance)) * 100
    
    # Overall pass/fail
    passes = pct_within >= 95  # At least 95% of points within tolerance
    
    results = {'step': step,
        'tolerance': f'{tolerance*100}%',
        'passes': passes,
        'points_within_tolerance': f'{pct_within: .4f}%',
        'num_original_points': len(y_original),
        'num_subsampled_points': len(y_subsampled),
        'data_range': f'{data_range: .4f}',  
        'max_normalized_error': f'{(np.max(normalized_errors)*100): .4f}%' }
    return results

for i, dframe in enumerate(rev_car_dataframes):
    mean_shifted_column = f"Signal{i+1} Zero Mean (m)"
    print(f"dataframe: df{i+1}")
    scipy_decimate_pct_check_results = percent_check_scipy_decimate(dframe, mean_shifted_column, step=50, tolerance=0.05)
    print(scipy_decimate_pct_check_results)
    print("")

##### Subsampling the Tapered DS Time Series

In [ ]:
def sub_tapered(tapered_data, decimation_factor):
    sub_tap_data = scipy.signal.decimate(tapered_data, decimation_factor, ftype='iir', zero_phase=True)
    return sub_tap_data

for i, dframe in enumerate(rev_car_dataframes):
    mean_shifted_column = f"Signal{i+1} Zero Mean (m)"
    time_column = f"Time{i+1} (s)"
    sub_tap_result = sub_tapered(apply_hann_window(dframe, mean_shifted_column), decimation_factor=50)
    # PLot:
    plt.figure(figsize=(12,4))
    plt.plot(dframe[time_column], dframe[mean_shifted_column], label=f"Full Data")
    plt.plot(dframe[time_column][::50], sub_tap_result, label=f"Subsampled/Tapered Data") # Make sure time step is same as decimation_factor
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude (m)")
    plt.grid(True, alpha=0.5)
    plt.title(f"Revving Car Engine Measurement {i+1} - Full and Subsampled/Tapered DS Data")
    plt.legend()
    plt.tight_layout()
    plt.show()

##### Fast Fourier Transform

In [ ]:
# Plotting FFT spectra showing tapered/subsampled data overlayed onto full DNS data for a given dataframe
def plot_fft(data, sr, fig1=None, fig2=None, fig3=None, name='', apply_window=True):
    """Plot FFT with windowing correction.
    data: array
    sr: Sampling rate
    apply_window : bool, says whether to apply Hann window (default True)"""
    n = len(data)
    
    # Apply window if requested
    if apply_window:
        window = np.hanning(n)
        windowed_data = data * window
    else:
        windowed_data = data
        coherent_gain=1.0
    
    # Compute FFT
    fft_values = scipy.fft.rfft(windowed_data)
    freqs = scipy.fft.rfftfreq(n, d=1/sr)
    
    # magnitudes WITHOUT coherent gain correction
    magnitudes = np.abs(fft_values) / n

    # Double the AC components (not DC and Nyquist)
    magnitudes[1:] *= 2.0

    # Undo doubling for Nyquist if even length
    if n % 2 == 0:
        magnitudes[-1] /= 2.0
    
    phases = np.angle(fft_values)
    
    # Create figures:
    if fig1 is None:
        fig1 = plt.figure(figsize=(10,6))
        plt.plot(freqs, magnitudes, linewidth=1, label=name)
        plt.title('FFT Magnitude Spectrum (Linear)', fontsize=16)
        plt.xlabel('Frequency (Hz)', fontsize=14)
        plt.ylabel('Magnitude', fontsize=14)
        plt.grid(True, which='major', alpha=0.5)
        plt.grid(True, which='minor', alpha=0.5)
        plt.minorticks_on()
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.tight_layout()
        if name:
            plt.legend(loc='upper right')
    else:
        plt.figure(fig1.number)
        plt.plot(freqs, magnitudes, linewidth=1, label=name)
        if name:
            plt.legend(loc='upper right')
    
    if fig2 is None:
        fig2 = plt.figure(figsize=(10, 6))
        plt.loglog(freqs[1:], magnitudes[1:], linewidth=1, label=name)
        plt.title('FFT Magnitude Spectrum (log)', fontsize=16)
        plt.xlabel('Frequency (Hz)', fontsize=14)
        plt.ylabel('Magnitude', fontsize=14)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.grid(True, which='major', alpha=0.5)
        plt.grid(True, which='minor', alpha=0.5)
        plt.tight_layout()
        if name:
            plt.legend(loc='upper right')
    else:
        plt.figure(fig2.number)
        plt.loglog(freqs[1:], magnitudes[1:], linewidth=1, label=name)
        plt.legend()
        if name:
            plt.legend(loc='upper right')
    
    if fig3 is None:
        fig3 = plt.figure(figsize=(10, 6))
        plt.plot(freqs[1:], phases[1:], linewidth=1, label=name)
        plt.title('Phase Spectrum', fontsize=16)
        plt.xlabel('Frequency (Hz)', fontsize=14)
        plt.ylabel('Phase (radians)', fontsize=14)
        plt.grid(True, alpha=0.5)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        #plt.xlim(-1, 50)
        plt.tight_layout()
        if name:
            plt.legend(loc='lower right', framealpha=0.5)
    else:
        plt.figure(fig3.number)
        plt.plot(freqs[1:], phases[1:], linewidth=1, label=name)
        if name:
            plt.legend(loc='lower right', framealpha=0.8)
    
    return fig1, fig2, fig3
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
for i, dframe in enumerate(rev_car_dataframes):
    mean_shifted_column = f"Signal{i+1} Zero Mean (m)"
    time_column = f"Time{i+1} (s)"
    decimation_factor=50 # step used for subsampling
    print(f"dataframe: df{i+1}")
    # For full data
    fig1, fig2, fig3 = plot_fft(dframe[mean_shifted_column].values, sr=25000, name='Full Data', apply_window=False)
    # For tapered/subsampled data
    sub_tap_results=[]
    sub_tap_result = sub_tapered(apply_hann_window(dframe, mean_shifted_column), decimation_factor=50)
    sub_tap_results.append(sub_tap_result)
    plot_fft(sub_tap_results[-1], sr=25000//decimation_factor, fig1=fig1, fig2=fig2, fig3=fig3, name='Subsampled/Tapered Data', apply_window=False)
    plt.show()

##### Continuous Wavelet Transform
- The Morlet wavelet, given by $\psi(t)=\exp(\frac{-t^2}{2})\cos(5t)$, is used for the wavelet transform
##### Full DS Data:

In [ ]:
for i, dframe in enumerate(rev_car_dataframes):
    mean_shifted_column=f"Signal{i+1} Zero Mean (m)"
    time_column = f"Time{i+1} (s)"
    data = dframe[mean_shifted_column].values
    #print(f"Data shape: {data.shape}")
    time_sec = dframe[time_column].values
    delta_t = time_sec[1] - time_sec[0] # time step in seconds
    #print(delta_t)

    scale_min = 1.0 / (10000 * delta_t)  # Small scale for high freq, use freq_max=10000
    scale_max = 1.0 / (10 * delta_t)  # Large scale for low freq, use freq_min=10
    scales = np.logspace(np.log10(scale_min), np.log10(scale_max), 60) 
    # Wavelet transform:
    coefficients, frequencies = pywt.cwt(data, scales, 'morl', sampling_period=delta_t)
    print(f"Frequency range: {frequencies.min():.2f} to {frequencies.max():.2f} Hz")
    print(f"Coefficients shape: {coefficients.shape}")
    print(f"Frequencies shape: {frequencies.shape}")
    print(f"Time_sec shape: {time_sec.shape}")
    # Smooth in time direction (axis=1), not frequency (axis=0)
    smoothed_coefficients = uniform_filter(np.abs(coefficients), size=(1, 200)) # increased time filter size from 50 to 200
    # Add small epsilon (if needed) to avoid log(0)
    #epsilon = 1e-10  # Small value to prevent log(0)
    #coeffs = (coefficients + epsilon)
    
    # Plot:
    plt.figure(figsize=(14, 6))
    plt.pcolormesh(time_sec, frequencies, np.log10(smoothed_coefficients), cmap='magma', shading='auto')#, vmin=-5)
    plt.colorbar(label='log\u2081\u2080(Magnitude)')
    plt.ylabel('Frequency (Hz)')
    plt.yscale('log')
    plt.xlabel('Time (s)')
    plt.title(f'Revving Car Engine Measurement {i+1} - Full DS Data')
    plt.show()

    # Try to make histograms ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    magnitudes = np.abs(smoothed_coefficients)
    magnitude_values = magnitudes.flatten()
    
    fig, ax = plt.subplots(figsize=(12, 4))
    plt.hist(magnitude_values, bins=50, density = True, facecolor='skyblue', edgecolor='grey')
    plt.title("Histogram of Scalogram Magnitudes")
    plt.xlabel("Magnitude")
    plt.ylabel("Probability Density")
    plt.yscale('log')
    plt.grid(axis='y', alpha=0.5)
    plt.axvline(x=np.mean(magnitude_values), color='r', linestyle='--', label=f'Mean: {np.mean(magnitude_values): .2e}')
    plt.axvline(x=np.median(magnitude_values), color='orange', linestyle='--', label=f'Median: {np.median(magnitude_values): .2e}')
    # get mode:
    hist_counts, bin_edges = np.histogram(magnitude_values, bins=50)
    mode_bin_index = np.argmax(hist_counts)
    # Mode is the center of the bin with the highest count:
    mode_value = (bin_edges[mode_bin_index] + bin_edges[mode_bin_index + 1]) / 2
    plt.axvline(x=mode_value, color='b', linestyle='--', label=f'Mode: {mode_value: .2e}')
    plt.legend()
    plt.show()
    # some other stats:
    print(ss.describe(magnitude_values))
    std_dev = np.std(magnitude_values)
    print(f"Standard Deviation: {std_dev:.4f}")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

##### Continuous Wavelet Transform
##### Tapered/Subsampled DS Data:

In [ ]:
for i, dframe in enumerate(rev_car_dataframes):
    mean_shifted_column=f"Signal{i+1} Zero Mean (m)"
    time_column = f"Time{i+1} (s)"
    sub_tap_data = sub_tapered(apply_hann_window(dframe, mean_shifted_column), decimation_factor=50)
    print(f"Data shape: {sub_tap_data.shape}")
    time_sec = dframe[time_column][::50].values # use step == decimation_factor 
    delta_t = time_sec[1] - time_sec[0] # time step in seconds
    #print(delta_t)

    scale_min = 1.0 / (250 * delta_t)  # Small scale for high freq, use freq_max=250
    scale_max = 1.0 / (10 * delta_t)  # Large scale for low freq, use freq_min=10
    scales = np.logspace(np.log10(scale_min), np.log10(scale_max), 60) 
    # Wavelet transform:
    coefficients, frequencies = pywt.cwt(sub_tap_data, scales, 'morl', sampling_period=delta_t)
    print(f"Frequency range: {frequencies.min():.2f} to {frequencies.max():.2f} Hz")
    print(f"Coefficients shape: {coefficients.shape}")
    print(f"Frequencies shape: {frequencies.shape}")
    print(f"Time_sec shape: {time_sec.shape}")
    # Smooth in time direction (axis=1), not frequency (axis=0)
    smoothed_coefficients = uniform_filter(np.abs(coefficients), size=(1, 25)) # changed filter size from (1,50) to (1,25)
    # Add small epsilon (if needed) to avoid log(0)
    #epsilon = 1e-10  # Small value to prevent log(0)
    #coeffs = (coefficients + epsilon)
    
    # Plot:
    plt.figure(figsize=(14, 6))
    plt.pcolormesh(time_sec, frequencies, np.log10(smoothed_coefficients), cmap='magma', shading='auto')
    plt.colorbar(label='log\u2081\u2080(Magnitude)')
    plt.ylabel('Frequency (Hz)')
    plt.yscale('log')
    plt.xlabel('Time (s)')
    plt.title(f'Revving Car Engine Measurement {i+1} - Full DS Data')
    plt.show()

    # Try to make histograms ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    magnitudes = np.abs(smoothed_coefficients)
    magnitude_values = magnitudes.flatten()
    
    fig, ax = plt.subplots(figsize=(12, 4))
    plt.hist(magnitude_values, bins=50, density = True, facecolor='skyblue', edgecolor='grey')
    plt.title("Histogram of Scalogram Magnitudes")
    plt.xlabel("Magnitude")
    plt.ylabel("Probability Density")
    plt.yscale('log')
    plt.grid(axis='y', alpha=0.5)
    plt.axvline(x=np.mean(magnitude_values), color='r', linestyle='--', label=f'Mean: {np.mean(magnitude_values): .2e}')
    plt.axvline(x=np.median(magnitude_values), color='orange', linestyle='--', label=f'Median: {np.median(magnitude_values): .2e}')
    # get mode:
    hist_counts, bin_edges = np.histogram(magnitude_values, bins=50)
    mode_bin_index = np.argmax(hist_counts)
    # Mode is the center of the bin with the highest count:
    mode_value = (bin_edges[mode_bin_index] + bin_edges[mode_bin_index + 1]) / 2
    plt.axvline(x=mode_value, color='b', linestyle='--', label=f'Mode: {mode_value: .2e}')
    plt.legend()
    plt.show()
    # some other stats:
    print(ss.describe(magnitude_values))
    std_dev = np.std(magnitude_values)
    print(f"Standard Deviation: {std_dev:.4f}")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")